# LangChain Agents Bootcamp - Financial Analysis with Gemini

Complete guide to building production-ready agents with memory, middleware, and streaming.

## Overview

This notebook covers:
- **Short-term Memory**: Persist conversation state with SQLite
- **Built-in Middleware**: 7 production patterns (summarization, limits, PII, etc.)
- **Context Engineering**: Manual and automatic context management
- **Structured Output**: Type-safe agent responses
- **Streaming Modes**: Real-time updates (`messages`, `updates`, `values`)
- **Context Caching**: Cost-efficient large document analysis

## Setup

Initialize model and tools for financial analysis.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [16]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage
from scripts import base_tools

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
# model = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")

system_prompt = """You are a financial analyst specializing in tech stocks.
Provide data-driven analysis with clear insights."""

## 1. Basic Agent

Create a simple agent with tools but no memory.

In [17]:
agent = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt=system_prompt
)

response = agent.invoke({
    "messages": [HumanMessage("What's Apple's current stock price?")]
})

print(response["messages"][-1].text)

Apple's current stock price (NASDAQ: AAPL) is $278.78. After hours, it is trading at $279.00, up by $0.22 (0.079%). This information was current as of market close on December 5, 7:58:32 PM GMT-5.


In [18]:
response

{'messages': [HumanMessage(content="What's Apple's current stock price?", additional_kwargs={}, response_metadata={}, id='42f32e26-a7fb-4ba5-8bce-2546af2059d7'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'web_search', 'arguments': '{"query": "Apple stock price"}'}, '__gemini_function_call_thought_signatures__': {'452baca8-ef24-470f-9c3e-28030129d670': 'Cv0BAXLI2nwyeKavXAeDl8SInVPkb2g+OAN1emFksKFuN17a1TKnbY+iWPAdYzX25ri5K9dVF3e9Db6ZkajJT1TroGSEaDTq25KTfAZn8rLhPehHGJzhcosdwNZUsNfpThjIMnJoSFqcdoKWDKd3fPv8kOCBV8u4RE3cFFDrBP3gqQBhMbm66K4NmGZDkrKfyHdiRh6hO0nZzx8MJdq8WxcZZLLjsrwbZzQYiWVrHCDjISTkPdEPUFdqsxwveUMUgqwjgXeITIorBhA3uXVpGsH+WeXIJCuatufLw6QdCVAz8BV8LudLIf2yRHk8ajgBPxFeLR/epGthtaJbtYQP/w=='}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--35f807e7-faab-43db-bc38-f249dfe31467-0', tool_calls=[{'name'

In [19]:
response["messages"][-1].content_blocks

[{'type': 'text',
  'text': "Apple's current stock price (NASDAQ: AAPL) is $278.78. After hours, it is trading at $279.00, up by $0.22 (0.079%). This information was current as of market close on December 5, 7:58:32 PM GMT-5.",
  'extras': {'signature': 'CpMHAXLI2nwxPqpAyNnEtQUXmK9dVyRIKaO8i9TyDNagOXH9qGjfydU4q8t4CI3I19W70NNvEOXaibjh1Gl/AkS3cfncYi9CdabujcscvMIafp2b8s2IFhpVCbveRY+dWAggGhpex2EsbxsIkM4le/1krvJiMEggEp8L8CtT1JiPL5wN1rDCStlDwtB2eHzorYiCoisfefY2ZfCqr4UcAblgfQPh5LPgMKqs+dUEOFqS69Bec9C7uaguF70/VseF82gilgkiSsHGSIxHbX+FTIEYbAYT1eyGBz9psHNDG9PQiTkbe0zwRI40b+0Iocj8lIKbepIXawhTuHDDvxtQinJfisKSyP3iXSxbmVGtuaQcclQg6TKqMJ5lfW9nbul74SD9Kr/2ouQaPvlXx/Xu0Y5/r+VRmP8YVYn+1O16aOjEwJ+Gqk40W1/pz2DZmYWx/5yXyAF8Khbqh2cksQYBe2aSK1MhvpzSZL2dF14Xij8NvEjrrWVQEfxz5ku04JTyCmXyuio2Ji4mlChEJU86meWOAB8wKuPF+XQN3RL0R/JjID0PORyNrb9CXuYPzt6+ra2GHXRTv5602zT5vh0kJ7z42KlD2rDhWOeHW4e4jbQ9Ch9C2NAPLYPUwiGMWvjzBz3xCKdLx2tSpmQP+++lKnWrUOgl/9yjj4Ol8wjqnKuDHza4DffLgyJtHbmyuXtrNeMEjxwrBxmqiexSUz6Sd0biKnCDxtMJ0YMw5Xm8x

In [22]:
response = agent.invoke({
    "messages": ["What's Apple's current stock price?"]
})
response

{'messages': [HumanMessage(content="What's Apple's current stock price?", additional_kwargs={}, response_metadata={}, id='2fc2902b-6097-4f5e-98bd-1654b13ba281'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'web_search', 'arguments': '{"query": "Apple stock price"}'}, '__gemini_function_call_thought_signatures__': {'88e0e055-f108-4f77-bbc0-857f4e8cbddc': 'CpQCAXLI2nyEu79uFQYCdItEnLDg8dgnP0QMQWTzqWY5mLQY8hyF9k4XFVXacHV90mgpFzHqDnwhmwEvz7m2MXlml5+jKp6qnF33bKEu1zcnrMbZbBxjRz2aIr1AlvFNnYtoT8tl7v/8BbW5ysMPEhff+iLs2giUopjqjCC2BHY5XbWwH2tDzzzUtEUos6p2+90VF1uW7LHZHVEqDL1gSOmslXADXOd4W29VQfubb4dzlRbzDecOY5v4ellm7LGCNwSBYfF48ioMiZEBabwCnIiinxqiPNd4g4OTftMY0s0NBnfIQD4Mm0sY7qYZmNT9mI+2SzbmbnpU0Si2NrW8gn3LHatYkTX9nT4zUePMvf8W1oHVXZcl'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--45638655-2390-4c3b-97dc-9d68060f

## 2. Short-term Memory with SQLite

Add conversation persistence using SQLite checkpointer. Agent remembers previous turns within a session.

In [23]:
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3

conn = sqlite3.connect("data/financial_agent.db", check_same_thread=False)
checkpointer = SqliteSaver(conn)

agent_memory = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt=system_prompt,
    checkpointer=checkpointer
)

In [ ]:
config = {"configurable": {"thread_id": "session_1"}}

# First turn
response = agent_memory.invoke({
    "messages": [HumanMessage("Search for Apple's latest earnings")]
}, config)
print(response["messages"][-1].text)

Turn 1: Apple reported strong fourth-quarter fiscal 2025 results, with revenue reaching $102.5 billion, an 8% increase year-over-year. Diluted earnings per share (EPS) were $1.85, up 13% year-over-year on an adjusted basis, beating analyst forecasts of $1.76. Services revenue hit a new all-time high, contributing to a record fiscal year revenue of $416 billion with double-digit EPS growth.

Key highlights include:
*   **Revenue:** $102.5 billion (vs. $101.69 billion forecast)
*   **EPS:** $1.85 (vs. $1.76 forecast)
*   **Services Revenue Growth:** 15% year-over-year.
*   **Fiscal Year Revenue:** $416 billion.

Challenges noted were supply chain constraints affecting iPhone models and a 4% decline in the Greater China market year-over-year. However, the company projects 10-12% revenue growth for the December quarter, with double-digit iPhone revenue growth. CEO Tim Cook highlighted AI integration across products, and CFO Kevin Parekh confirmed increased AI investments.


In [26]:
# Second turn - remembers context
response = agent_memory.invoke({
    "messages": [HumanMessage("How does this compare to last quarter?")]
}, config)
print(response["messages"][-1].text)

Compared to the previous quarter (Q3 2025), Apple's performance in Q4 2025 showed significant growth:

*   **Revenue:**
    *   **Q4 2025:** $102.5 billion
    *   **Q3 2025:** $94.04 billion
    *   **Comparison:** Revenue increased by approximately $8.46 billion (9.0%) from Q3 to Q4.

*   **Diluted Earnings Per Share (EPS):**
    *   **Q4 2025:** $1.85
    *   **Q3 2025:** $1.57
    *   **Comparison:** EPS increased by $0.28 (17.8%) from Q3 to Q4.

In summary, Apple experienced strong sequential growth in both revenue and EPS from the third to the fourth fiscal quarter of 2025.


In [27]:
response

{'messages': [HumanMessage(content="Search for Apple's latest earnings", additional_kwargs={}, response_metadata={}, id='a2d64139-937c-4976-a84b-49e39946b3db'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'web_search', 'arguments': '{"query": "Apple latest earnings"}'}, '__gemini_function_call_thought_signatures__': {'a23ee1ea-c53b-44a7-9e50-db8749b21239': 'CsgBAXLI2nzgynGhDgtm6iiMdIzRJRfEcNpzqW/+ctu520gV2jQvlb/4ZpRmnCwW73RdQsEbebXjTzohCWYCdDyskWniSHbMTdSV3SZYIOYzHukyPBtRCsSsxzsoG/7WS1rZyePHYDvjq3q5tyzy0jDEwa1Vba+w0vDPx2BpBH5AGqI88CszshIKPD1ayaTS5E+LKOgC/krTbuSw/1rc/7R3EikbyqaQnzYJo4zEUZfpK7XdOqqIu9ETvNq/KuxjvZpsvIPF2n/Mv10='}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--d667d027-d69a-4861-a148-3a69e22ff3d8-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'Apple latest earnings'}, 'id': 'a2

## Built-in Middleware

### 1. Middleware: Summarization

Automatically summarize old messages when history grows too long.

**Use Case**: Long conversations that exceed context windows.

In [ ]:
from langchain.agents.middleware import SummarizationMiddleware

agent_summary = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt=system_prompt,
    checkpointer=checkpointer,
    middleware=[
        SummarizationMiddleware(
            model="gemini-2.5-flash",
            trigger={"messages": 10},
            keep={"messages": 4}
        )
    ]
)

In [ ]:
config = {"configurable": {"thread_id": "summary_session"}}

queries = [
    "Search for Apple stock",
    "What about Microsoft?",
    "Compare their P/E ratios",
    "Tech sector outlook?",
    "Market trends?"
]

for query in queries:
    response = agent_summary.invoke(
        {"messages": [HumanMessage(query)]},
        config
    )
    print(f"Q: {query}\nA: {response['messages'][-1].content}\n")

state = agent_summary.get_state(config)
print(f"Total messages: {len(state.values['messages'])}")

## 4. Middleware: Model Call Limit

Limit the number of model calls per request to prevent runaway costs.

**Exit Behaviors**: `"end"` (stop) or `"continue"` (proceed without model)

In [ ]:
from langchain.agents.middleware import ModelCallLimitMiddleware

agent_limit = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt=system_prompt,
    middleware=[
        ModelCallLimitMiddleware(
            run_limit=3,
            exit_behavior="end"
        )
    ]
)

response = agent_limit.invoke({
    "messages": [HumanMessage("Analyze Apple stock")]
})
print("Limited calls:", response["messages"][-1].content)

## 5. Middleware: Tool Call Limit

Limit the number of tool calls to control external API usage.

In [ ]:
from langchain.agents.middleware import ToolCallLimitMiddleware

agent_tool_limit = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt=system_prompt,
    middleware=[
        ToolCallLimitMiddleware(
            run_limit=2,
            exit_behavior="continue"
        )
    ]
)

response = agent_tool_limit.invoke({
    "messages": [HumanMessage("Check Apple and sector trends")]
})
print("Tool limited:", response["messages"][-1].content)

## 6. Middleware: Model Fallback

Automatically fallback to a different model if primary fails.

In [ ]:
from langchain.agents.middleware import ModelFallbackMiddleware

agent_fallback = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt=system_prompt,
    middleware=[
        ModelFallbackMiddleware("gemini-2.0-flash-exp")
    ]
)

## 7. Middleware: PII Detection

Automatically detect and redact/mask personally identifiable information.

**Strategies**: `"redact"` (remove), `"mask"` (replace with ***)

In [ ]:
from langchain.agents.middleware import PIIMiddleware

agent_pii = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt=system_prompt,
    middleware=[
        PIIMiddleware("email", strategy="redact", apply_to_input=True),
        PIIMiddleware("credit_card", strategy="mask", apply_to_input=True)
    ]
)

response = agent_pii.invoke({
    "messages": [HumanMessage("Email test@example.com for Apple updates")]
})
print("PII handled:", response["messages"][-1].content)

## 8. Middleware: Todo List

Track and manage multi-step tasks within the agent.

In [ ]:
from langchain.agents.middleware import TodoListMiddleware

agent_todo = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt=system_prompt,
    checkpointer=checkpointer,
    middleware=[TodoListMiddleware()]
)

config = {"configurable": {"thread_id": "todo_session"}}

response = agent_todo.invoke({
    "messages": [HumanMessage("Analyze Apple revenue and compare competitors")]
}, config)
print("Todo response:", response["messages"][-1].content)

## 9. Middleware: Context Editing

Automatically clean up tool call history after a certain token threshold.

In [ ]:
from langchain.agents.middleware import ContextEditingMiddleware, ClearToolUsesEdit

agent_context = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt=system_prompt,
    checkpointer=checkpointer,
    middleware=[
        ContextEditingMiddleware(
            edits=[
                ClearToolUsesEdit(
                    trigger=100000,
                    keep=3
                )
            ]
        )
    ]
)

## Context Engineering

Manual control over conversation state.

## 10. Manual Context Editing: Delete Messages

Programmatically trim conversation history to save tokens.

In [ ]:
config = {"configurable": {"thread_id": "session_1"}}
state = agent_memory.get_state(config)
print(f"Messages before: {len(state.values['messages'])}")

# Keep only system message and last 2 messages
messages = list(state.values['messages'])
edited_messages = [messages[0]] + messages[-2:]

agent_memory.update_state(config, {"messages": edited_messages})

state = agent_memory.get_state(config)
print(f"Messages after: {len(state.values['messages'])}")

## 11. Combined Middleware

Stack multiple middleware for production-ready agents.

In [ ]:
agent_complete = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt=system_prompt,
    checkpointer=checkpointer,
    middleware=[
        SummarizationMiddleware(model="gemini-2.5-flash", trigger={"messages": 12}, keep={"messages": 5}),
        ModelCallLimitMiddleware(run_limit=5),
        ToolCallLimitMiddleware(run_limit=3),
        PIIMiddleware("email", strategy="redact", apply_to_input=True),
        TodoListMiddleware()
    ]
)

## Advanced Features

## 12. Structured Output

Return type-safe Pydantic models from agent responses.

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional

class FinancialAnalysis(BaseModel):
    """Structured financial analysis."""
    company: str = Field(description="Company name")
    stock_symbol: str = Field(description="Stock ticker")
    current_price: Optional[str] = Field(description="Current price")
    analysis: str = Field(description="Brief analysis")
    recommendation: str = Field(description="Buy/Hold/Sell")

In [ ]:
model_structured = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
model_with_tools = model_structured.bind_tools([base_tools.web_search])
structured_llm = model_with_tools.with_structured_output(FinancialAnalysis)

agent_structured = create_agent(
    model=structured_llm,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt="Provide structured financial analysis.",
    checkpointer=checkpointer
)

In [ ]:
config = {"configurable": {"thread_id": "structured_session"}}

response = agent_structured.invoke({
    "messages": [HumanMessage("Analyze Apple stock")]
}, config)

if "structured_response" in response:
    analysis = response["structured_response"]
    print(f"Company: {analysis.company}")
    print(f"Symbol: {analysis.stock_symbol}")
    print(f"Analysis: {analysis.analysis}")
    print(f"Recommendation: {analysis.recommendation}")

## Streaming Modes

Three streaming modes for real-time agent updates:
- **`messages`**: Stream individual messages as they're generated
- **`updates`**: Stream state updates after each step
- **`values`**: Stream complete state values

## 13. Streaming: Messages Mode

Stream messages token-by-token as they're generated.

In [ ]:
agent_stream = create_agent(
    model=model,
    tools=[base_tools.web_search, base_tools.get_weather],
    system_prompt=system_prompt,
    checkpointer=checkpointer
)

config = {"configurable": {"thread_id": "stream_1"}}

print("\n=== Stream: Messages ===")
for chunk in agent_stream.stream({
    "messages": [HumanMessage("Quick Apple stock update")]
}, config):
    print(chunk)
    print("---")

## 14. Streaming: Updates Mode

Stream state updates after each agent step (tool calls, model responses).

In [ ]:
config = {"configurable": {"thread_id": "stream_2"}}

print("\n=== Stream: Updates ===")
for chunk in agent_stream.stream({
    "messages": [HumanMessage("Tech sector outlook?")]
}, config, stream_mode="updates"):
    print(chunk)
    print("---")

## 15. Streaming: Values Mode

Stream complete state snapshots (useful for monitoring message count).

In [ ]:
config = {"configurable": {"thread_id": "stream_3"}}

print("\n=== Stream: Values ===")
for chunk in agent_stream.stream({
    "messages": [HumanMessage("Compare Apple and Microsoft")]
}, config, stream_mode="values"):
    if "messages" in chunk:
        print(f"State: {len(chunk['messages'])} messages")
    print("---")